In [147]:
# 전략은 이렇다. 
# 1. 우선 각 지역은 지금처럼 get_info 함수에 넣어 놓고 일괄적으로 입력하되 
# 2. 이 크롤러는 특수하게 다른것 처럼 구 아래 예속된것이 아니라  별개로 존제한다. 
# 3  다만 메인 크롤러에서 각 구의 도서관 코드를 나열하고 포문 돌며 크롤러 호출했듯이 여기서도 똑같이 하지는 못한다. 
# 4. 한 페이지에 여러개 지역의 여러이름의 도서관이 섞여 있어서 안된다.       
# 5. 이경우 크롤링중 뽑은도서관 코드 바로 넣어줌. 

# 내가 수기로 입력하지 않은 도서관 새로생기면? -> 무비 저장 movie save 과정에서 도서관 못찾으면아래 저장과정 못하게 하자. 


import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib import parse



import time
def get_url_data(url, max_tries=10):
    for n in range(max_tries):
        try:
            return requests.get(url)
        except requests.exceptions.RequestException:
            if n == max_tries - 1:
                raise
            time.sleep(10)

year = "2019"
month = "06"

# 페이지 번호 뽑기위해 초벌 크롤 
url = "https://lib.sen.go.kr/lib/board/index.do?plan_date="+year+"-"+month+"&menu_idx=12&manage_idx=1121&board_idx=0&rowCount=50&plan_year="+year+"&plan_month="+month+"&viewPage=1&search_type=title%2Bcontent"
request = get_url_data(url)
response = request.text
soup = BeautifulSoup(response, 'lxml')



# 페이지 부터 뽑고 그것을 포문 돌아 보겠슴. 
pages = [] 

page_button_parent = soup.find("div", {"id": "board_paging"})
if page_button_parent:
    page_button = page_button_parent.select("a.paginate_button")
    if page_button:
        for i in range(len(page_button)):
            pages.append(i+1)
    

print(pages)
count = 0 
for page in pages:
    url = "https://lib.sen.go.kr/lib/board/index.do?plan_date="+year+"-"+month+"&menu_idx=12&manage_idx=1121&board_idx=0&rowCount=50&plan_year="+year+"&plan_month="+month+"&viewPage="+str(page)+"&search_type=title%2Bcontent"
    request = get_url_data(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')



    movie_tables = soup.select('div.search-results > div.row') # 영화 없으면 아예 아래 포문 안들어가게 하겠슴.

    for movie_table in movie_tables:
        count+=1


        title = ""  # 제목
        when = ""  # 일시
        when_date_year = 0
        when_date_month = 0
        when_date_day = 0
        when_time_hour = 0
        when_time_minuite = 0
        runtime = 0  # 런타임
        place = ""  # 장소

        libGroup = ""


        # 제목 따로 a테그라 먼저 찾겠슴 
        title_pre = movie_table.select_one('div.box > div.item > div.bif > a.name')

        if title_pre:
            title = title_pre.get("title")
            
            if re.findall('\(.+\)\s*(.+)',title):       # (독립영화) 이월 
                title_before_pre = re.findall('\(.+\)\s*(.+)',title)
                if title_before_pre:
                    title = title_before_pre[0]    
            elif re.findall('(.+)\s*\(.+\)',title):       # 이월 (독립영화),(더빙),(자막) 등 빼주기위함. 
                title_before_pre = re.findall('(.+)\s*\(.+\)',title)
                if title_before_pre:
                    title = title_before_pre[0]      
            elif re.findall('\[.+\]\s*(.+)',title):        # [강서스크린] 웃는남자 
                title_before_pre = re.findall('\[.+\]\s*(.+)',title)
                if title_before_pre:
                    title = title_before_pre[0]    
            elif re.findall('(.+)\s*\[.+\]',title):        #  웃는남자 [강서스크린]
                title_before_pre = re.findall('(.+)\s*\[.+\]',title)
                if title_before_pre:
                    title = title_before_pre[0]     
            elif re.findall('용산시네마여행_(.+)',title):        #  용산시네마여행_
                title_before_pre = re.findall('용산시네마여행_(.+)',title)
                if title_before_pre:
                    title = title_before_pre[0]
            elif re.findall('독립영화상영_(.+)',title):        #  독립영화상영_ 
                title_before_pre = re.findall('독립영화상영_(.+)',title)
                if title_before_pre:
                    title = title_before_pre[0] 
            elif re.findall('힐링뮤직박스_(.+)',title):        #  힐링뮤직박스_
                title_before_pre = re.findall('힐링뮤직박스_(.+)',title)
                if title_before_pre:
                    title = title_before_pre[0]
            print(f'title: {title}')
        else:
            print('영화제목없음')
            continue# 타이틀 없으면 다음 영화로 넘어감. 


        # 나머지정보들 묶여서 한번에 크롤링 
        item_pres = movie_table.select('div.box > div.item > div.bif > ul.con2 > li')

        found_libgroup =False #아래 코드 뽑을때 한번만 get하기 위함.
        for item_pre in item_pres:

            # 도서관 코드 뽑는것 추가. 
            if not found_libgroup and item_pre.get('class'):
                libGroup = item_pre.get('class')[0]
                found_libgroup = True

            item_name_pre = item_pre.get_text()
            if "상영일시" in item_name_pre:

                when = item_name_pre
#                     print(f'when: {when}')

                when_date_year_pre = re.findall('(\d\d\d\d)', when)
                if when_date_year_pre:
                    when_date_year = when_date_year_pre[0]
                when_date_month_pre = re.findall('\d\d\d\d\s*-\s*(\d{1,2})', when)
                if when_date_month_pre:
                    when_date_month = when_date_month_pre[0]
                when_date_day_pre = re.findall('\d\d\d\d\s*-\s*\d{1,2}\s*-\s*(\d{1,2})', when)
                if when_date_day_pre:
                    when_date_day = when_date_day_pre[0]

                # 18:00 이런식으로 표현시 
                if re.findall('(\d{1,2})\s*:', when):
                    when_time_hour_pre= re.findall('(\d{1,2})\s*:', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                    when_time_minuite_pre = re.findall('\d{1,2}\s*:\s*(\d{1,2})\s*', when)
                    if when_time_minuite_pre:
                        when_time_minuite = when_time_minuite_pre[0]
                # 18시 30분 이런식으로 표현시 
                elif re.findall('\d+\s*시\s*(\d+)\s*분', when):
                    when_time_hour_pre = re.findall('(\d+)\s*시\s*\d+\s*분', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]
                    when_time_minuite_pre = re.findall('\d+\s*시\s*(\d+)\s*분', when)
                    if when_time_minuite_pre:
                        when_time_minuite = when_time_minuite_pre[0]

                # 18시 이런식으로 표현될땐 ~
                elif re.findall('(\d+)\s*시', when):
                    when_time_hour_pre = re.findall('(\d+)\s*시', when)
                    if when_time_hour_pre:
                        when_time_hour = when_time_hour_pre[0]

                print(f'when_date_year: {when_date_year}')
                print(f'when_date_month: {when_date_month}')
                print(f'when_date_day: {when_date_day}')
                print(f'when_time_hour: {when_time_hour}')
                print(f'when_time_minuite: {when_time_minuite}')


            elif "상영장소" in item_name_pre:
                place_pre = item_name_pre.split(':')
                if place_pre:
                    place = place_pre[1]
                    print(f'place: {place}')

            elif "상영시간" in item_name_pre:
                runtime_pre  =re.findall('(\d+)', item_name_pre)
                if runtime_pre:
                    runtime = runtime_pre[0]
                    print(f'runtime: {runtime}')


        print(f'libGroup : {libGroup}')

        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')


[1, 2, 3]
title: 마이펫 다이노소어
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : nslib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 마션
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : dblib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 피터와 늑대
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : splib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 샤샤의 북극 대모험
when_date_year: 2019
when_date_month: 06
when_date_day: 29
when_time_hour: 14
when_time_minuite: 30
libGroup : gdlib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 찰스디킨스의 비밀서재
when_date_year: 2019
when_date_month: 06
when_date_day: 29
when_time_hour: 14
when_time_minuite: 00
libGroup : nslib
@@@@@@@@@@@@@@@@@@@@@@@

title: 이월
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 0
when_time_minuite: 0
libGroup : ahyon
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 드래곤길들이기2
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : gnlib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 미운오리새끼의 모험
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : gplib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 국제시장
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : djlib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 몬스터호텔 3
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : ddmlib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

title: 랭고
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
libGroup : grlib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 해리포터와 죽음의 성물 2
when_date_year: 2019
when_date_month: 06
when_date_day: 29
when_time_hour: 0
when_time_minuite: 0
libGroup : yslib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 인천상륙작전
when_date_year: 2019
when_date_month: 06
when_date_day: 28
when_time_hour: 14
when_time_minuite: 00
libGroup : jdlib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 월터의 상상은 현실이 된다.
when_date_year: 2019
when_date_month: 06
when_date_day: 26
when_time_hour: 14
when_time_minuite: 30
libGroup : yslib
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 국민 DJ 이종환의 7080 라디오 1
when_date_year: 2019
when_date_month: 06
when_date_day: 26
when_time_hour: 14
when_time_minuite: 00
libGroup : yslib
@@@@@@@@@@@@@@@

In [106]:
# get_url_data써서 request실패해도 10번 재시도함. 테스트 해봄. 



import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib import parse


def get_url_data(url, max_tries=10):
    for n in range(max_tries):
        try:
            return requests.get(url)
        except requests.exceptions.RequestException:
            if n == max_tries - 1:
                raise
            time.sleep(40)


year = "2019"
month = "06"

url = "https://lib.sen.go.kr/lib/board/index.do?plan_date="+year+"-"+month+"&menu_idx=12&manage_idx=1121&board_idx=0&rowCount=50&plan_year="+year+"&plan_month="+month+"&viewPage=1&search_type=title%2Bcontent"
request = get_url_data(url)
response = request.text
soup = BeautifulSoup(response, 'lxml')

movie_tables = soup.select('div.search-results > div.row') # 영화 없으면 아예 아래 포문 안들어가게 하겠슴.

for movie_table in movie_tables:

    title = ""  # 제목
    when = ""  # 일시
    when_date_year = 0
    when_date_month = 0
    when_date_day = 0
    when_time_hour = 0
    when_time_minuite = 0
    runtime = 0  # 런타임
    place = ""  # 장소


    # 제목 따로 a테그라 먼저 찾겠슴 
    title_pre = movie_table.select_one('div.box > div.item > div.bif > a.name')
    if title_pre:
        title = title_pre.get_text(strip=True)
        print(f'title: {title}')
    else:
        print('영화제목없음')
        continue# 타이틀 없으면 다음 영화로 넘어감. 


    # 나머지정보들 묶여서 한번에 크롤링 
    item_pres = movie_table.select('div.box > div.item > div.bif > ul.con2 > li')

    for item_pre in item_pres:
        item_name_pre = item_pre.get_text()

        if "상영일시" in item_name_pre:

            when = item_name_pre
            print(f'when: {when}')

            when_date_year_pre = re.findall('(\d\d\d\d)', when)
            if when_date_year_pre:
                when_date_year = when_date_year_pre[0]
            when_date_month_pre = re.findall('\d\d\d\d\s*-\s*(\d{1,2})', when)
            if when_date_month_pre:
                when_date_month = when_date_month_pre[0]
            when_date_day_pre = re.findall('\d\d\d\d\s*-\s*\d{1,2}\s*-\s*(\d{1,2})', when)
            if when_date_day_pre:
                when_date_day = when_date_day_pre[0]

            # 18:00 이런식으로 표현시 
            if re.findall('(\d{1,2})\s*:', when):
                when_time_hour_pre= re.findall('(\d{1,2})\s*:', when)
                if when_time_hour_pre:
                    when_time_hour = when_time_hour_pre[0]
                when_time_minuite_pre = re.findall('\d{1,2}\s*:\s*(\d{1,2})\s*', when)
                if when_time_minuite_pre:
                    when_time_minuite = when_time_minuite_pre[0]
            # 18시 30분 이런식으로 표현시 
            elif re.findall('\d+\s*시\s*(\d+)\s*분', when):
                when_time_hour_pre = re.findall('(\d+)\s*시\s*\d+\s*분', when)
                if when_time_hour_pre:
                    when_time_hour = when_time_hour_pre[0]
                when_time_minuite_pre = re.findall('\d+\s*시\s*(\d+)\s*분', when)
                if when_time_minuite_pre:
                    when_time_minuite = when_time_minuite_pre[0]

            # 18시 이런식으로 표현될땐 ~
            elif re.findall('(\d+)\s*시', when):
                when_time_hour_pre = re.findall('(\d+)\s*시', when)
                if when_time_hour_pre:
                    when_time_hour = when_time_hour_pre[0]

            print(f'when_date_year: {when_date_year}')
            print(f'when_date_month: {when_date_month}')
            print(f'when_date_day: {when_date_day}')
            print(f'when_time_hour: {when_time_hour}')
            print(f'when_time_minuite: {when_time_minuite}')


        elif "상영장소" in item_name_pre:
            place_pre = item_name_pre.split(':')
            if place_pre:
                place = place_pre[1]
                print(f'place: {place}')

        elif "상영시간" in item_name_pre:
            runtime_pre  =re.findall('(\d+)', item_name_pre)
            if runtime_pre:
                runtime = runtime_pre[0]
                print(f'runtime: {runtime}')

    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

        
        

title: 마이펫 다이노소어
when: 상영일시 : 2019-06-30
										14:00
									
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
runtime: 93
place:  2층 문화누리실
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 마션
when: 상영일시 : 2019-06-30
										14:00
									
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
runtime: 144
place:  4층 시청각실
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 피터와 늑대
when: 상영일시 : 2019-06-30
										14:00
									
when_date_year: 2019
when_date_month: 06
when_date_day: 30
when_time_hour: 14
when_time_minuite: 00
runtime: 49
place:  아트홀(지하1층)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
title: 샤샤의 북극 대모험
when: 상영일시 : 2019-06-29
										14:30
									
when_date_year: 2019
when_date_month: 06
when_date_day: 29
when_time_hour: 14
when_time_minuite: 30
runtime: 81
place:  음향영상실